In [14]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [15]:
train_dir = "../Dataset/train/"
test_dir = "../Dataset/test/"

In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


In [16]:
datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 2637 images belonging to 2 classes.


In [18]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 660 images belonging to 2 classes.


In [19]:
train_features = base_model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)

83/83 [==============================] - 99s 1s/step


In [20]:
test_features = base_model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)

21/21 [==============================] - 25s 1s/step


In [21]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features, train_generator.labels)

KNeighborsClassifier()

In [22]:
predictions = knn.predict(test_features)


In [25]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(test_generator.labels, predictions)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(test_generator.labels, predictions)
print(report)

# Calculate confusion matrix
confusion_mat = confusion_matrix(test_generator.labels, predictions)
print("Confusion Matrix:")
print(confusion_mat)

Accuracy: 0.5287878787878788
              precision    recall  f1-score   support

           0       0.56      0.60      0.58       360
           1       0.48      0.45      0.46       300

    accuracy                           0.53       660
   macro avg       0.52      0.52      0.52       660
weighted avg       0.53      0.53      0.53       660

Confusion Matrix:
[[215 145]
 [166 134]]


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score


param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}


knn = KNeighborsClassifier()


grid_search = GridSearchCV(knn, param_grid, cv=5)


grid_search.fit(train_features, train_generator.labels)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


knn_best = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'],
                                weights=best_params['weights'],
                                p=best_params['p'])
knn_best.fit(train_features, train_generator.labels)


predictions = knn_best.predict(test_features)


accuracy = accuracy_score(test_generator.labels, predictions)
print("Accuracy:", accuracy)

report = classification_report(test_generator.labels, predictions)
print(report)

Best Parameters: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Best Score: 0.5119487091024093
Accuracy: 0.5287878787878788
              precision    recall  f1-score   support

           0       0.56      0.60      0.58       360
           1       0.48      0.45      0.46       300

    accuracy                           0.53       660
   macro avg       0.52      0.52      0.52       660
weighted avg       0.53      0.53      0.53       660

